In [1]:
import pandas as pd
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage
from tensorflow.keras import callbacks
from tensorflow.keras import Sequential, Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

2025-06-05 15:36:34.881594: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 15:36:34.899825: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 15:36:35.032478: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 15:36:35.146737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749134195.238517   91617 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749134195.26

In [2]:
# Variables
BUCKET_NAME="skin_scan_mohnatz"
GCP_PROJECT = "skin-scan-461716"
BQ_DATASET = "skin_scan"

In [3]:
def get_data_from_bq(
        gcp_project:str,
        query:str,
        data_has_header=True
    ) -> pd.DataFrame:
    """
    Retrieve `query` data from BigQuery
    """
    client = bigquery.Client(project=gcp_project)
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()
    return df

query_metadata = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.train_metadata
    """

In [4]:
metadata = get_data_from_bq(f"{GCP_PROJECT}", query_metadata)
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,index
0,HAM_0006706,ISIC_0031725,nv,histo,NaN,female,abdomen,8410
1,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen,2388
2,HAM_0002610,ISIC_0030606,vasc,consensus,0.0,female,abdomen,2389
3,HAM_0003872,ISIC_0028903,nv,consensus,0.0,female,abdomen,9329
4,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen,1035


In [5]:
import sys
import os

# Add parent directory to Python path
sys.path.append(os.path.abspath(".."))

# Now you can import the module
import preprocessing

In [6]:
X_preprocessed, y_preprocessed = preprocessing.preprocess_metadata(metadata,split=True)

In [7]:
X_preprocessed.head()

,passthrough__image_id,age__age,cat__sex_male,cat__localization_acral,cat__localization_back,cat__localization_chest,cat__localization_ear,cat__localization_face,cat__localization_foot,cat__localization_genital,cat__localization_hand,cat__localization_lower extremity,cat__localization_neck,cat__localization_scalp,cat__localization_trunk,cat__localization_upper extremity
7765,ISIC_0024306,0.529412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5914,ISIC_0024307,0.588235,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8234,ISIC_0024308,0.647059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7525,ISIC_0024309,0.470588,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3461,ISIC_0024310,0.705882,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y_preprocessed.head()

,dx_akiec,dx_bcc,dx_bkl,dx_df,dx_mel,dx_nv,dx_vasc
7765,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5914,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8234,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7525,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3461,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
from google.cloud import storage
from PIL import Image
import io
import matplotlib.pyplot as plt

# Set up the client
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blobs = bucket.list_blobs(prefix="train_all_images/")
images = [blob.name for blob in blobs if blob.name.lower().endswith(".jpg")]

In [12]:
image_name = []
resized_array = []

for index, image in enumerate(images):
    blob = bucket.blob(image)
    image_bytes = blob.download_as_bytes()
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    img = img.resize((96, 96))
    resized_array.append(np.array(img))
    image_name.append(os.path.basename(image).split('.')[0])
    print(f"(Resized image number {index}")

(Resized image number 0
(Resized image number 1
(Resized image number 2
(Resized image number 3
(Resized image number 4
(Resized image number 5
(Resized image number 6
(Resized image number 7
(Resized image number 8
(Resized image number 9
(Resized image number 10
(Resized image number 11
(Resized image number 12
(Resized image number 13
(Resized image number 14
(Resized image number 15
(Resized image number 16
(Resized image number 17
(Resized image number 18
(Resized image number 19
(Resized image number 20
(Resized image number 21
(Resized image number 22
(Resized image number 23
(Resized image number 24
(Resized image number 25
(Resized image number 26
(Resized image number 27
(Resized image number 28
(Resized image number 29
(Resized image number 30
(Resized image number 31
(Resized image number 32
(Resized image number 33
(Resized image number 34
(Resized image number 35
(Resized image number 36
(Resized image number 37
(Resized image number 38
(Resized image number 39
(Resized i

In [30]:
test_df = pd.DataFrame({
    'image_id': image_name,
    'resized_array': resized_array
})

In [31]:
test_df.head()

,image_id,resized_array
0,ISIC_0024306,"[[[228, 131, 133], [230, 130, 137], [231, 124,..."
1,ISIC_0024307,"[[[216, 121, 133], [218, 122, 137], [218, 125,..."
2,ISIC_0024308,"[[[246, 173, 181], [245, 171, 175], [247, 171,..."
3,ISIC_0024309,"[[[206, 118, 134], [209, 120, 136], [210, 120,..."
4,ISIC_0024310,"[[[72, 33, 38], [70, 32, 38], [69, 31, 38], [6..."


In [35]:
full_df = metadata.merge(test_df,how="left",on="image_id")
full_df.head()

,image_id,target,resized_array
0,ISIC_0024306,4,"[[[228, 131, 133], [230, 130, 137], [231, 124,..."
1,ISIC_0024307,4,"[[[216, 121, 133], [218, 122, 137], [218, 125,..."
2,ISIC_0024308,4,"[[[246, 173, 181], [245, 171, 175], [247, 171,..."
3,ISIC_0024309,4,"[[[206, 118, 134], [209, 120, 136], [210, 120,..."
4,ISIC_0024310,6,"[[[72, 33, 38], [70, 32, 38], [69, 31, 38], [6..."


In [36]:
full_df.drop(columns=[col for col in ["image_id"] if col in full_df.columns],inplace=True)

In [37]:
full_df.head()

,target,resized_array
0,4,"[[[228, 131, 133], [230, 130, 137], [231, 124,..."
1,4,"[[[216, 121, 133], [218, 122, 137], [218, 125,..."
2,4,"[[[246, 173, 181], [245, 171, 175], [247, 171,..."
3,4,"[[[206, 118, 134], [209, 120, 136], [210, 120,..."
4,6,"[[[72, 33, 38], [70, 32, 38], [69, 31, 38], [6..."


In [49]:
X = full_df["resized_array"]

In [50]:
y = full_df["target"]

In [51]:
X_np = np.array(X)

In [52]:
X_np.shape

(10015,)

In [55]:
X_np[0].shape

(96, 96, 3)

In [72]:
X_ready = []
for row in X_np:
    X_ready.append(X[0])

In [73]:
X_ready = np.array(X_ready)

In [74]:
X_ready

array([[[[228, 131, 133],
         [230, 130, 137],
         [231, 124, 133],
         ...,
         [235, 140, 147],
         [227, 133, 133],
         [225, 134, 133]],

        [[230, 132, 134],
         [229, 130, 133],
         [230, 126, 130],
         ...,
         [233, 140, 147],
         [230, 138, 140],
         [227, 138, 136]],

        [[229, 133, 134],
         [230, 130, 133],
         [229, 124, 129],
         ...,
         [235, 142, 151],
         [235, 143, 149],
         [231, 141, 139]],

        ...,

        [[211, 121, 125],
         [211, 121, 125],
         [210, 119, 123],
         ...,
         [219, 148, 136],
         [215, 144, 134],
         [211, 142, 129]],

        [[210, 120, 120],
         [210, 121, 122],
         [211, 123, 124],
         ...,
         [218, 147, 134],
         [215, 144, 134],
         [213, 142, 131]],

        [[209, 119, 121],
         [209, 118, 119],
         [211, 124, 125],
         ...,
         [218, 148, 135],
        

In [75]:
y.shape

(10015,)

In [76]:
from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y)
y_cat

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [77]:
X_ready.shape

(10015, 96, 96, 3)

In [78]:
X_train, X_test,y_train, y_test = train_test_split(X_ready, y_cat, test_size=0.3, random_state=42)

In [79]:
X_train.shape

(7010, 96, 96, 3)

In [82]:
X_train = X_train/255

In [83]:
X_test = X_test/255

# Model Architecture

In [95]:
model = Sequential()

model.add(Input(shape=(96, 96, 3)))
model.add(layers.Conv2D(16, (6, 6), padding='same', activation="relu"))
model.add(layers.Conv2D(16, (4, 4), padding='same', activation="relu"))
model.add(layers.Conv2D(32, (3, 3), padding='same', activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

In [96]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 96, 96, 16)     │         1,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 96, 96, 16)     │         4,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 96, 96, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 294912)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 20)             │     5,898,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 7)              │           147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,909,323 (22.54 MB)

 Trainable params: 5,909,323 (22.54 MB)

 Non-trainable params: 0 (0.00 B)

In [97]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy","recall"])

In [98]:
es = EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train,
          epochs=100,  # Use early stopping in practice
          batch_size=32,
          verbose=1,
          validation_split = 0.2,
          callbacks=[es])